In [159]:
import csv, re, requests, json

In [160]:
len(Place.objects.all())

0

In [161]:
for x in Place.objects.all():
    x.delete()
for x in Institution.objects.all():
    x.delete()

In [162]:
# fetch data directly from github
# see http://stackoverflow.com/questions/35371043/use-python-requests-to-download-csv
url = 'https://raw.githubusercontent.com/gfranzini/digEds_cat/master/digEds_cat_spatial.csv'
with requests.Session() as s:
    download = s.get(url)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    datalist = list(cr)

In [163]:
#load local copy of data
#file = "data/digEds_cat_spatial.csv"
#with open(file, 'r', encoding='utf-8') as data:
    #reader = csv.reader(data)
    #datalist = list(reader)

In [164]:
#iterats over rows in csv. sends for each lng-lat a request to geonames 
# api to fetch matching places (and the countries they are located in) 
# creates Place objects and Institution objects, relats and stores them in db.
root = "http://api.geonames.org/findNearbyPlaceNameJSON?lat="
geoname_jsons = []
for row in datalist:
    if row[1] != "" and row[1] != "Latitude":
        lat, lng = format(float(row[1]), '.3f'), format(float(row[2]), '.3f')
        params = "{}&lng={}&username=digitalarchiv".format(lat, lng)
        url = root + params
        r = requests.get(url)
        x =  r.json()
        if len(x["geonames"]) >= 1:
            place_dict = x["geonames"][0]
            temp_country, _ = Place.objects.get_or_create(geonames_id=place_dict["countryId"],
                                                         name=place_dict["countryName"])
            temp_country.save()
            temp_country.place_type = "country"
            temp_country.save()
            temp_place, _ = Place.objects.get_or_create(geonames_id=place_dict["geonameId"],
                                                        name=place_dict["name"],
                                                        place_type="city",
                                                        lat=place_dict["lat"],
                                                        lng=place_dict["lng"])
            temp_place.part_of = temp_country
            temp_place.save()
            temp_inst, _ = Institution.objects.get_or_create(name=row[0])
            temp_inst.lat = row[1]
            temp_inst.lng = row[2]
            temp_inst.save()
            temp_inst.place = temp_place
            temp_inst.save()

In [165]:
print("Countries created: ", len(Place.objects.filter(place_type="country")))
print("Places created: ", len(Place.objects.filter(place_type="city")))
print("Institutions created: ", len(Institution.objects.all()) )

Countries created:  23
Places created:  116
Institutions created:  124


In [166]:
for x in Place.objects.all():
    print("{}, {}, {}, {}".format(x.name, x.place_type, x.geonames_id, x.part_of))

Australia, country, 2077456, None
Canberra, city, 2172517, Australia
Austria, country, 2782113, None
Andräviertel, city, 6941547, Austria
Innere Stadt, city, 2775260, Austria
Spain, country, 2510769, None
Santsoena, city, 3110032, Spain
Belgium, country, 2802361, None
Kerkeveld, city, 2794549, Belgium
Ghent, city, 2797656, Belgium
Canada, country, 6251999, None
Barkers Point, city, 5893060, Canada
Montgomery, city, 6077007, Canada
United States, country, 6252001, None
New Brunswick, city, 5101717, United States
Sand Pits, city, 6140443, Canada
Croatia, country, 3202326, None
Zagreb- Donji Grad, city, 10286588, Croatia
Denmark, country, 2623032, None
Nyhavn, city, 2615979, Denmark
Egypt, country, 357994, None
Saint Catherine, city, 8521413, Egypt
Finland, country, 660013, None
Kruununhaka, city, 650744, Finland
France, country, 3017382, None
Paris 12 Ancien - Quartier Saint-Marcel, city, 9843572, France
La Mouche, city, 3007992, France
Auteuil, city, 6545275, France
Austerlitz, city, 65